In [1]:
import pandas as pd
import numpy as np
from numpy import NaN
from sklearn.preprocessing import OneHotEncoder

In [2]:
#There are 4 datasets that we are looking to combine for analysis. 
#Before combining the 4 datasets, for each dataset, we need to FIRST, check for valid entries 
#by comparing the entries with the possible valid entries found in the data dictionary 
#provided with the datasets, SECOND, we need to check for duplicate occurences of the 
#Incident Number as we will need to have UNIQUE incident numbers to join on. 


In [3]:
##Importing Data Dictionary. Will do Data clearning on columns by comparing entriess with the "data dictionary" to catch an spelling mistakes or errors"
DataDictionary = pd.read_csv("/Users/nerdbear/Downloads/1. pca_national_human_wildlife_conflict_data_dictionary.csv", encoding='utf8', converters={'Data_Field': str.strip})


In [4]:
DataDictionary.head()

,Data_Field,Champ_de_la_donnée,Data_Value,Valeur_de_la_donnée,Value_Description,Description_de_la_valeur
0,Activity Type,Type d'activité,Backpacking – Multiday Trips,Randonnée pédestre – excursion de plusieurs jours,Backpacking – Multiday Trips,Randonnée pédestre – excursion de plusieurs jours
1,Activity Type,Type d'activité,Beach Recreation,Activitée de plage,Beach Recreation,Activitée de plage
2,Activity Type,Type d'activité,Boating - Coastal/Marine,Navigation - côtière/marin,Boating - Coastal/Marine,Navigation - côtière/marin
3,Activity Type,Type d'activité,Boating - Commercial,Navigation - Commerciale,Boating - Commercial,Navigation - Commerciale
4,Activity Type,Type d'activité,Boating - Motorized Pleasure Craft,Navigation de plaisance – embarcation motorisée,Boating - Motorized Pleasure Craft,Navigation de plaisance – embarcation motorisée


In [5]:
#Drop french columns
DataDictionary = DataDictionary.drop(["Champ_de_la_donnée", "Valeur_de_la_donnée", "Description_de_la_valeur"], axis=1)


In [6]:
DataDictionary.head()

,Data_Field,Data_Value,Value_Description
0,Activity Type,Backpacking – Multiday Trips,Backpacking – Multiday Trips
1,Activity Type,Beach Recreation,Beach Recreation
2,Activity Type,Boating - Coastal/Marine,Boating - Coastal/Marine
3,Activity Type,Boating - Commercial,Boating - Commercial
4,Activity Type,Boating - Motorized Pleasure Craft,Boating - Motorized Pleasure Craft


In [7]:
DataDictionary["Data_Field"].unique()

array(['Activity Type', 'Animal Attractant', 'Animal Behaviour',
       'Animal Health Status', 'Cause of Animal Health Status',
       'Deterrents/Projectiles used', 'Field Unit', 'Incident Type',
       'Month', 'Protected Heritage Area', 'Reason for Animal Behaviour',
       'Response to Deterrent', 'Response Type', 'Species Common Name'],
      dtype=object)

In [8]:
#1 of 4 datasets
##


In [9]:
Activities = pd.read_csv("/Users/nerdbear/Downloads/3. pca-human-wildlife-coexistence-activities-detailed-records-2010-2021.csv", encoding='cp1252')
#Note, encoding='cp1252' needed to be specified in order to read .csv withour parser errors


In [10]:
Activities.head()

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Activity Type
0,2017-HWC-0005-YKLLFU-0001,2017-08-01,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving
1,2017-HWC-0005-YKLLFU-0002,2017-09-07,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Camping - Backcountry
2,2017-HWC-0005-YKLLFU-0003,2017-07-08,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving
3,2017-HWC-0005-YKLLFU-0004,2017-06-23,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving
4,2017-HWC-0005-YKLLFU-0006,2017-06-28,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving


In [11]:
Activities.shape

(66284, 5)

In [12]:
Activities.dtypes

Incident Number            object
Incident Date              object
Field Unit                 object
Protected Heritage Area    object
Activity Type              object
dtype: object

In [13]:
#First we are going to clean the data to ensure valid entries on the string values by comparing them to the data dictionary. 


In [14]:
#Checking to see how many values in Activity Type do not match values in dictionary.
Activities["Activity Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Activity Type"]).sum()


60363

In [15]:
#Shows how many are False, therefore how many activity types are not in the dictionary. 
Activities.shape[0] - Activities["Activity Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Activity Type"].unique()).sum()
#There are 5921 entries that do not match values found in dictionary. Next lines of code look into this more to determine if any can be replaced with valid fields


5921

In [16]:
#Add column to dataframe that indicates which values match dictionary (True) and which do not (False)
Activities["Activity_Type_Dict"] = Activities["Activity Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Activity Type"].unique()
)
Activities.head()

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Activity Type,Activity_Type_Dict
0,2017-HWC-0005-YKLLFU-0001,2017-08-01,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving,True
1,2017-HWC-0005-YKLLFU-0002,2017-09-07,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Camping - Backcountry,True
2,2017-HWC-0005-YKLLFU-0003,2017-07-08,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving,True
3,2017-HWC-0005-YKLLFU-0004,2017-06-23,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving,True
4,2017-HWC-0005-YKLLFU-0006,2017-06-28,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving,True


In [17]:
#Print values that do not match dictionary to see which need to be replaced. 
Activities["Activity Type"][Activities["Activity_Type_Dict"]== False].unique()


array(['Docking - TINP Only', nan, 'Avoidance', 'Niking / Walking',
       'Biking / Walking', 'Wiking / Walking', 'Sightseeing',
       'Camping-Frontcountry', 'Hiking/Walking',
       'Resource Harvesting - Hunting', 'Hiking', 'Picknicking / BBQ',
       'Picnicking/BBQ'], dtype=object)

In [18]:
#printing all activity types from dictionary to see which best match the errors listed above
DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Activity Type"].unique()


array(['Backpacking – Multiday Trips', 'Beach Recreation',
       'Boating - Coastal/Marine', 'Boating - Commercial',
       'Boating - Motorized Pleasure Craft', 'Bush Party',
       'Camping - Backcountry', 'Camping - Frontcountry',
       'Camping - Huts and Lodges', 'Camping - Winter Frontcountry',
       'Canoeing - Coastal', 'Canoeing - Flatwater',
       'Canoeing - Swiftwater', 'Canyon exploration -Winter',
       'Canyoneering', 'Caving', 'Climbing - Bouldering',
       'Climbing - Mountaineering', 'Climbing - Technical Rock',
       'Climbing - Waterfall  Ice', 'Commercial Transportation Operation',
       'Cycling', 'Cycling - Mountain Biking',
       'Cycling - Road/Shared Path', 'Cycling - Winter', 'Dog Walking',
       'Dogsledding', 'Domestic Residence Activity', 'Driving',
       'Field Sports', 'Fishing',
       'Flight - BASE Jumping/ Proximity Flying',
       'Flight - Hang-gliding/Parapenting', 'Flight - Helicopter',
       'Flight - HETS', 'Flight - Sightseeing/Sit

In [19]:
#Replacing Activity Types that were mis-entered with their proper type, if none was obvious from data dictionary, placing "Unknown"
#Not replacing NaN values with "Unknown". Will look at missing values closer later after splitting and before modelling. 
 
Activities["Activity Type"] = Activities["Activity Type"].replace({"Docking - TINP Only": "Unknown", "Avoidance": "Unknown", "Niking / Walking": 'Hiking / Walking',"Biking / Walking": 'Hiking / Walking', "Camping-Frontcountry": 'Camping - Frontcountry', "Hiking/Walking": 'Hiking / Walking', "Resource Harvesting - Hunting": 'Resource Harvesting - Hunting/Fishing/Gathering/Trapping', "Hiking": 'Hiking / Walking', "Picknicking / BBQ": "Picnicking / BBQ", "Picnicking/BBQ": "Picnicking / BBQ", 'Wiking / Walking': 'Hiking / Walking', 'Sightseeing':'Heritage Activity - Sightseeing'})

In [20]:
#Counts number of True values
Activities["Activity Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Activity Type"].unique())


0        True
1        True
2        True
3        True
4        True
         ... 
66279    True
66280    True
66281    True
66282    True
66283    True
Name: Activity Type, Length: 66284, dtype: bool

In [21]:
#Checking again (after replacement) to see if any Activity Type values still do NOT match dictionary
Activities.shape[0] -  Activities["Activity Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Activity Type"].unique()).sum()



5885

In [22]:
Activities["Activity Type"].isna().sum()
#All remaining are missing values. 

5885

In [23]:
#Checking to see how many values in Protected Heritage Area do not match values in dictionary
Activities.shape[0] - Activities["Protected Heritage Area"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Protected Heritage Area"].unique()
).sum()
#There are none that are not in dictionary. No replacements needed. 

0

In [24]:
#Checking to see how many values in Field Unit do not match values in dictionary
Activities.shape[0] - Activities["Field Unit"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Field Unit"].unique()
).sum()
#There are none that are not in dictionary. No replacements needed. 

0

In [25]:
#Drop the columns I added during cleaning that are no longer needed
Activities = Activities.drop(["Activity_Type_Dict"], axis=1)
Activities.head()

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Activity Type
0,2017-HWC-0005-YKLLFU-0001,2017-08-01,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving
1,2017-HWC-0005-YKLLFU-0002,2017-09-07,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Camping - Backcountry
2,2017-HWC-0005-YKLLFU-0003,2017-07-08,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving
3,2017-HWC-0005-YKLLFU-0004,2017-06-23,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving
4,2017-HWC-0005-YKLLFU-0006,2017-06-28,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving


In [26]:
#Next, we're looking for duplicate occurances of the incident number to ensure our final compiled data has UNIQUE incident numbers. 


In [27]:
Act_subset = Activities[["Incident Number", "Incident Date", "Field Unit", "Protected Heritage Area"]]
duplicate_Act_subset = Act_subset.duplicated(keep=False)
sum(duplicate_Act_subset)

4051

In [28]:
duplicate_Act_Inc_Num = Activities.duplicated(subset="Incident Number", keep=False)
sum(duplicate_Act_Inc_Num)

4051

In [29]:
sum(duplicate_Act_Inc_Num)==sum(duplicate_Act_subset)


True

In [30]:
#Conclusion, The Activity Type column is the column that differs between rows - all other columns are identical when the incident number duplicates


In [31]:
#I would like to encode Acitivity Type so each distinct activity type is it's own column with a binary indicator (0 for no and 1 for yes) if a given activity type was involved in a given incident number.



In [32]:
#Count distinct values in Activity Type
Activities["Activity Type"].nunique()

88

In [33]:
encoder = OneHotEncoder(handle_unknown='ignore')

In [34]:
encoder_df = pd.DataFrame(encoder.fit_transform(Activities[["Activity Type"]]).toarray())

In [35]:
encoder_df.columns = encoder.get_feature_names_out(["Activity Type"])

In [36]:
encoder_df.head()

,Activity Type_Backpacking – Multiday Trips,Activity Type_Beach Recreation,Activity Type_Boating - Coastal/Marine,Activity Type_Boating - Commercial,Activity Type_Boating - Motorized Pleasure Craft,Activity Type_Bush Party,Activity Type_Camping - Backcountry,Activity Type_Camping - Frontcountry,Activity Type_Camping - Huts and Lodges,Activity Type_Camping - Winter Frontcountry,...,Activity Type_Swimming - Coastal,Activity Type_Swimming - Facilities,Activity Type_Swimming - Flat Water,Activity Type_Swimming - Swiftwater,Activity Type_Townsite Activity,Activity Type_Tram/Ski Lift/Gondola,Activity Type_Tubing / River Drifting,Activity Type_Unknown,Activity Type_Via-Ferrata,Activity Type_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
Activities_encoded = Activities.join(encoder_df)

In [38]:
Activities_encoded.head()

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Activity Type,Activity Type_Backpacking – Multiday Trips,Activity Type_Beach Recreation,Activity Type_Boating - Coastal/Marine,Activity Type_Boating - Commercial,Activity Type_Boating - Motorized Pleasure Craft,...,Activity Type_Swimming - Coastal,Activity Type_Swimming - Facilities,Activity Type_Swimming - Flat Water,Activity Type_Swimming - Swiftwater,Activity Type_Townsite Activity,Activity Type_Tram/Ski Lift/Gondola,Activity Type_Tubing / River Drifting,Activity Type_Unknown,Activity Type_Via-Ferrata,Activity Type_nan
0,2017-HWC-0005-YKLLFU-0001,2017-08-01,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-HWC-0005-YKLLFU-0002,2017-09-07,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Camping - Backcountry,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-HWC-0005-YKLLFU-0003,2017-07-08,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-HWC-0005-YKLLFU-0004,2017-06-23,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-HWC-0005-YKLLFU-0006,2017-06-28,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Driving,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
Activities_encoded.drop('Activity Type', axis = 1, inplace=True)

In [40]:
Activities_encoded.head()

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Activity Type_Backpacking – Multiday Trips,Activity Type_Beach Recreation,Activity Type_Boating - Coastal/Marine,Activity Type_Boating - Commercial,Activity Type_Boating - Motorized Pleasure Craft,Activity Type_Bush Party,...,Activity Type_Swimming - Coastal,Activity Type_Swimming - Facilities,Activity Type_Swimming - Flat Water,Activity Type_Swimming - Swiftwater,Activity Type_Townsite Activity,Activity Type_Tram/Ski Lift/Gondola,Activity Type_Tubing / River Drifting,Activity Type_Unknown,Activity Type_Via-Ferrata,Activity Type_nan
0,2017-HWC-0005-YKLLFU-0001,2017-08-01,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-HWC-0005-YKLLFU-0002,2017-09-07,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-HWC-0005-YKLLFU-0003,2017-07-08,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-HWC-0005-YKLLFU-0004,2017-06-23,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-HWC-0005-YKLLFU-0006,2017-06-28,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
Activities_encoded[Activities_encoded.columns[4:105]]

,Activity Type_Backpacking – Multiday Trips,Activity Type_Beach Recreation,Activity Type_Boating - Coastal/Marine,Activity Type_Boating - Commercial,Activity Type_Boating - Motorized Pleasure Craft,Activity Type_Bush Party,Activity Type_Camping - Backcountry,Activity Type_Camping - Frontcountry,Activity Type_Camping - Huts and Lodges,Activity Type_Camping - Winter Frontcountry,...,Activity Type_Swimming - Coastal,Activity Type_Swimming - Facilities,Activity Type_Swimming - Flat Water,Activity Type_Swimming - Swiftwater,Activity Type_Townsite Activity,Activity Type_Tram/Ski Lift/Gondola,Activity Type_Tubing / River Drifting,Activity Type_Unknown,Activity Type_Via-Ferrata,Activity Type_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
Activities2 = Activities_encoded[Activities_encoded.columns[4:105]].groupby([Activities['Incident Number'], Activities["Incident Date"], Activities["Field Unit"], Activities["Protected Heritage Area"]]).apply(sum).reset_index()
Activities2.head()

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Activity Type_Backpacking – Multiday Trips,Activity Type_Beach Recreation,Activity Type_Boating - Coastal/Marine,Activity Type_Boating - Commercial,Activity Type_Boating - Motorized Pleasure Craft,Activity Type_Bush Party,...,Activity Type_Swimming - Coastal,Activity Type_Swimming - Facilities,Activity Type_Swimming - Flat Water,Activity Type_Swimming - Swiftwater,Activity Type_Townsite Activity,Activity Type_Tram/Ski Lift/Gondola,Activity Type_Tubing / River Drifting,Activity Type_Unknown,Activity Type_Via-Ferrata,Activity Type_nan
0,2017-HWC-0005-YKLLFU-0001,2017-08-01,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-HWC-0005-YKLLFU-0002,2017-09-07,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-HWC-0005-YKLLFU-0003,2017-07-08,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-HWC-0005-YKLLFU-0004,2017-06-23,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-HWC-0005-YKLLFU-0006,2017-06-28,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
# The way I've merged the encoded activity type columns using the (sum) function means that instead of being a binary indicator, it is now a count of the number of times each activity was recorded per incident *IF there was more than one of the same activity type on a single incident number)



In [44]:
#Confirming whether the new dataset has any duplicate incident numbers
duplicate_Act2_Inc_Num = Activities2.duplicated(subset="Incident Number", keep=False)
sum(duplicate_Act2_Inc_Num)

0

In [45]:
dup_Activities = Activities[duplicate_Act_subset]

In [46]:
#Cross checking to ensure correct number of rows remain.
#Number of rows in Original Dataset, minus (number of rows in duplicates subset minus number of UNIQUE rows in duplicates subset) == Number of final rows in new subset. 
Activities.shape[0] - (dup_Activities.shape[0] - dup_Activities["Incident Number"].nunique()) == Activities2.shape[0]


True

In [47]:
#(In other words, I want to ensure that our new dataset has the same number of Unique incident numbers as our original dataset)
Activities["Incident Number"].nunique() == Activities2["Incident Number"].nunique()

True

In [48]:
# 2 of 4 datasets
##

In [49]:
Animals = pd.read_csv("/Users/nerdbear/Downloads/4. pca-human-wildlife-coexistence-animals-involved-detailed-records-2010-2021.csv", encoding='cp1252')


In [50]:
Animals.head()

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used,Animal Response to Deterrents
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,Surprise,Prey animal (natural),Presence of Officer/Person,NaN
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,NaN,NaN,NaN,NaN
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,NaN,Prey animal (natural),NaN,NaN
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,NaN,NaN,NaN,NaN
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,0,NaN,NaN,NaN,NaN,Grain,NaN,NaN


In [51]:
Animals.shape


(73655, 14)

In [52]:
Animals.dtypes

Incident Number                  object
Incident Date                    object
Field Unit                       object
Protected Heritage Area          object
Incident Type                    object
Species Common Name              object
Sum of Number of Animals          int64
Animal Health Status             object
Cause of Animal Health Status    object
Animal Behaviour                 object
Reason for Animal Behaviour      object
Animal Attractant                object
Deterrents Used                  object
Animal Response to Deterrents    object
dtype: object

In [53]:
#First we are going to clean the data to ensure valid entries on the string values by comparing them to the data dictionary. 

In [54]:
#Checking to see how many values in Field Unit do not match values in dictionary.
Animals["Field Unit"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Field Unit"]).sum()


73655

In [55]:
#Shows how many are False, therefore how many Field Units are not in the dictionary. 
Animals.shape[0] - Animals["Field Unit"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Field Unit"].unique()).sum()
#There are none that are not in dictionary. No replacements needed. 

0

In [56]:
#Checking to see how many values in Protected Heritage Area do not match values in dictionary.
Animals["Protected Heritage Area"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Protected Heritage Area"]).sum()


73655

In [57]:
#Shows how many are False, therefore how many Field Units are not in the dictionary. 
Animals.shape[0] - Animals["Protected Heritage Area"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Protected Heritage Area"].unique()).sum()
#There are none that are not in dictionary. No replacements needed. 

0

In [58]:
#Checking to see how many values in Incident Type do not match values in dictionary.
Animals["Incident Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Incident Type"]).sum()


73655

In [59]:
#Shows how many are False, therefore how many FIncident Type are not in the dictionary. 
Animals.shape[0] - Animals["Incident Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Incident Type"].unique()).sum()
#There are none that are not in dictionary. No replacements needed. 

0

In [60]:
#Checking to see how many values in Species Common Name do not match values in dictionary.
Animals["Species Common Name"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Species Common Name"]).sum()


73653

In [61]:
#Shows how many are False, therefore how many Species Common Name are not in the dictionary. 
Animals.shape[0] - Animals["Species Common Name"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Species Common Name"].unique()).sum()
#There are 2 that are not in dictionary. Replacements needed. 

2

In [62]:
#Add column to dataframe that indicates which values match dictionary (True) and which do not (False)
Animals["Species Common Name_Dict"] = Animals["Species Common Name"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Species Common Name"].unique())
Animals.head()


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used,Animal Response to Deterrents,Species Common Name_Dict
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,Surprise,Prey animal (natural),Presence of Officer/Person,NaN,True
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,NaN,NaN,NaN,NaN,True
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,NaN,Prey animal (natural),NaN,NaN,True
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,NaN,NaN,NaN,NaN,True
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,0,NaN,NaN,NaN,NaN,Grain,NaN,NaN,True


In [63]:
#Print values that do not match dictionary to see which need to be replaced. 
Animals["Species Common Name"][Animals["Species Common Name_Dict"]== False].unique()

array(['Banff Spring Snail', 'Eurasian red squirrel'], dtype=object)

In [64]:
#printing all species common name from dictionary to see which best match the errors listed above
DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Species Common Name"].unique()

array(['American Coot', 'American Dipper', 'American Dog Tick',
       'American eel', 'American Kestrel', 'American Robin',
       'American sand lance', 'American Toad', 'American Tree Sparrow',
       'Ant', 'Arctic Fox', 'Arctic Ground Squirrel', 'Atlantic Cod',
       'Atlantic Halibut', 'Atlantic Herring', 'Atlantic Salmon',
       'Atlantic White-sided Dolphin', 'Badger', 'Bald Eagle',
       'Banff Springs Snail', 'Bank Swallow', 'Barn Swallow',
       'Barred Owl', 'Basking Shark', 'Bearded Seal', 'Beaver',
       'Belted Kingfisher', 'Beluga Whale', 'Big Brown Bat', 'Big Skate',
       'Bighorn Sheep', 'Black Bear', 'Black Duck', 'Black Oystercatcher',
       'Black Rat', 'Black Scoter', 'Black Swift', 'Black Widow Spider',
       'Black-billed Murrelet', 'Black-footed Albatross',
       'Black-footed Ferret', 'Black-tailed deer',
       'Black-tailed prairie dog', 'Black-throated Sparrow',
       "Blanding's Turtle", 'Blue Grouse', 'Blue Jay', 'Blue Shark',
       'Blue Whal

In [65]:
#Replacing Activity Types that were mis-entered with their proper type, if none was obvious from data dictionary, placing "Unknown"
Animals["Species Common Name"] = Animals["Species Common Name"].replace({"Banff Spring Snail":"Banff Springs Snail","Eurasian red squirrel": "Red Squirrel"})


In [66]:
#Recehcking how many are False, therefore how many Species Common Name are not in the dictionary. 
Animals.shape[0] - Animals["Species Common Name"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Species Common Name"].unique()).sum()
#There are none that are not in dictionary. No replacements needed. 

0

In [67]:
#Checking to see how many values in Animal Health Status do not match values in dictionary.
Animals["Animal Health Status"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Animal Health Status"]).sum()


41477

In [68]:
#Shows how many are False, therefore how many Animal Health Status are not in the dictionary. 
Animals.shape[0] - Animals["Animal Health Status"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Animal Health Status"].unique()).sum()
#There are lots that are not in dictionary. Will look at these deeper to determine if replacement needed

32178

In [69]:
#Add column to dataframe that indicates which values match dictionary (True) and which do not (False)
Animals["Animal Health Status_Dict"] = Animals["Animal Health Status"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Animal Health Status"].unique())
Animals.head()

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used,Animal Response to Deterrents,Species Common Name_Dict,Animal Health Status_Dict
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,Surprise,Prey animal (natural),Presence of Officer/Person,NaN,True,True
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,NaN,NaN,NaN,NaN,True,True
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,NaN,Prey animal (natural),NaN,NaN,True,True
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,NaN,NaN,NaN,NaN,True,True
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,0,NaN,NaN,NaN,NaN,Grain,NaN,NaN,True,False


In [70]:
#Print values that do not match dictionary to see which need to be replaced. 
Animals["Animal Health Status"][Animals["Animal Health Status_Dict"]== False].unique()

array([nan, 'Not Applicable'], dtype=object)

In [71]:
#printing all activity types from dictionary to see which best match the errors listed above
DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Animal Health Status"].unique()

array(['Dead', 'Healthy', 'Injured', 'Not Located', 'Orphaned', 'Other',
       'Sick', 'Unknown'], dtype=object)

In [72]:
#Not replacing NaN values with "Unknown". 
#Will look at missing values closer later after after splitting and before modelling. 
#I'm going to keep the "Not applicable" entries because those are realistic valid 
#entries and not typos/errors. 
#No replacements or changes to make. 


In [73]:
#Checking to see how many values in Cause of Animal Health Status do not match values in dictionary.
Animals["Cause of Animal Health Status"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Cause of Animal Health Status"]).sum()


13080

In [74]:
#Shows how many are False, therefore how many Cause of Animal Health Status are not in the dictionary. 
Animals.shape[0] - Animals["Cause of Animal Health Status"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Cause of Animal Health Status"].unique()).sum()
#There are plenty that are not in dictionary. No replacements needed. 

60575

In [75]:
#Add column to dataframe that indicates which values match dictionary (True) and which do not (False)
Animals["Cause of Animal Health Status_Dict"] = Animals["Cause of Animal Health Status"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Cause of Animal Health Status"].unique())
Animals.head()


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used,Animal Response to Deterrents,Species Common Name_Dict,Animal Health Status_Dict,Cause of Animal Health Status_Dict
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,Surprise,Prey animal (natural),Presence of Officer/Person,NaN,True,True,False
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,NaN,NaN,NaN,NaN,True,True,True
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,NaN,Prey animal (natural),NaN,NaN,True,True,False
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,NaN,NaN,NaN,NaN,True,True,True
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,0,NaN,NaN,NaN,NaN,Grain,NaN,NaN,True,False,False


In [76]:
#Print values that do not match dictionary to see which need to be replaced. 
Animals["Cause of Animal Health Status"][Animals["Cause of Animal Health Status_Dict"]== False].unique()


array([nan, 'Not Applicable'], dtype=object)

In [77]:
#printing all activity types from dictionary to see which best match the errors listed above
DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Cause of Animal Health Status"].unique()

array(['Collision', 'Defence of Life/Property - public', 'Disease',
       'Drowned', 'Entangle-Entrapment', 'Hunting - Trapping',
       'Indigenous Harvest', 'Intraspecific Competition',
       'Management Destruction', 'Natural Mortality', 'Other', 'Poaching',
       'Poisoned', 'Predation', 'Starvation', 'Unknown'], dtype=object)

In [78]:
#Not replacing NaN values with "Unknown". 
#Will look at missing values closer later after splitting and before modelling. 
#I'm going to keep the "Not applicable" entries because those are realistic valid 
#entries and not typos/errors. 
#No replacements or changes to make. 

In [79]:
#Checking to see how many values do not match values in dictionary.
Animals["Animal Behaviour"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Animal Behaviour"]).sum()


45674

In [80]:
#Shows how many are False, therefore how many activity types are not in the dictionary. 
Animals.shape[0] - Animals["Animal Behaviour"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Animal Behaviour"].unique()).sum()
#There are several entries that do not match values found in dictionary. Next lines of code will look to see if any can be replaced


27981

In [81]:
#Add column to dataframe that indicates which values match dictionary (True) and which do not (False)
Animals["Animal Behaviour_Dict"] = Animals["Animal Behaviour"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Animal Behaviour"].unique())
Animals.head()


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used,Animal Response to Deterrents,Species Common Name_Dict,Animal Health Status_Dict,Cause of Animal Health Status_Dict,Animal Behaviour_Dict
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,Surprise,Prey animal (natural),Presence of Officer/Person,NaN,True,True,False,True
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,NaN,NaN,NaN,NaN,True,True,True,False
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,NaN,Prey animal (natural),NaN,NaN,True,True,False,False
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,NaN,NaN,NaN,NaN,True,True,True,False
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,0,NaN,NaN,NaN,NaN,Grain,NaN,NaN,True,False,False,False


In [82]:
#Print values that do not match dictionary to see which need to be replaced. 
Animals["Animal Behaviour"][Animals["Animal Behaviour_Dict"]== False].unique()

array([nan, 'Stress'], dtype=object)

In [83]:
#printing all activity types from dictionary to see which best match the errors listed above
DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Animal Behaviour"].unique()

array(['Avoidance', 'Bluff Charge', 'Chase', 'Contact-People',
       'Contact-Pet', 'Contact-Property', 'Curious', 'Curious Approach',
       'Dive', 'Escort (Follow-Flank)', 'Indifferent to People/Vehicles',
       'Intense Staring', 'Not Applicable', 'Other',
       'Physical or Aggressive Display', 'Predatory Approach',
       'Presence - Wildlife Exclusion Zones', 'Secretive', 'Unaware',
       'Unknown', 'Unyielding (refuse to give ground)', 'Vocalization'],
      dtype=object)

In [84]:
#Replacing Stress with other as Stress is not a valid entry per the dictionary, there is only 1 occurence and "other" fits best.  
Animals["Animal Behaviour"] = Animals["Animal Behaviour"].replace({"Stress": "Other"})


In [85]:
#All other values that do not match dictionary are missing. No more replacements needed. 

In [86]:
#Checking to see how many values do not match values in dictionary.
Animals["Reason for Animal Behaviour"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Reason for Animal Behaviour"]).sum()


24849

In [87]:
#Shows how many are False, therefore how many activity types are not in the dictionary. 
Animals.shape[0] - Animals["Reason for Animal Behaviour"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Reason for Animal Behaviour"].unique()).sum()
#There are lots of entries that do not match values found in dictionary. Next lines of code will look to see if any can be replaced


48806

In [88]:
#Add column to dataframe that indicates which values match dictionary (True) and which do not (False)
Animals["Reason for Animal Behaviour_Dict"] = Animals["Reason for Animal Behaviour"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Reason for Animal Behaviour"].unique())
Animals.head()

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used,Animal Response to Deterrents,Species Common Name_Dict,Animal Health Status_Dict,Cause of Animal Health Status_Dict,Animal Behaviour_Dict,Reason for Animal Behaviour_Dict
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,Surprise,Prey animal (natural),Presence of Officer/Person,NaN,True,True,False,True,True
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,NaN,NaN,NaN,NaN,True,True,True,False,False
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,NaN,Prey animal (natural),NaN,NaN,True,True,False,False,False
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,NaN,NaN,NaN,NaN,True,True,True,False,False
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,0,NaN,NaN,NaN,NaN,Grain,NaN,NaN,True,False,False,False,False


In [89]:
#Print values that do not match dictionary to see which need to be replaced. 
Animals["Reason for Animal Behaviour"][Animals["Reason for Animal Behaviour_Dict"]== False].unique()


array([nan, 'Not applicable', 'Entangle-Entrapment'], dtype=object)

In [90]:
#printing all activity types from dictionary to see which best match the errors listed above
DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Reason for Animal Behaviour"].unique()


array(['Defence of Food', 'Defence of Mate', 'Defence of Space',
       'Defence of Young', 'Disease', 'Food Conditioned', 'Food Reward',
       'Habituation', 'Not Applicable', 'Predator Avoidance', 'Predatory',
       'Presence of Domestic Animal', 'Starvation', 'Stress', 'Surprise',
       'Unknown'], dtype=object)

In [91]:
Animals.loc[Animals["Reason for Animal Behaviour"]== "Entangle-Entrapment"]


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used,Animal Response to Deterrents,Species Common Name_Dict,Animal Health Status_Dict,Cause of Animal Health Status_Dict,Animal Behaviour_Dict,Reason for Animal Behaviour_Dict
9220,YNP2012-0188,2012-07-27,"Lake Louise, Yoho and Kootenay Field Unit",Yoho National Park of Canada,Human Wildlife Interaction,Black Bear,1,NaN,NaN,Presence - Wildlife Exclusion Zones,Entangle-Entrapment,Unknown,NaN,NaN,True,False,False,True,False


In [92]:
#There is only one occurence of "Entangle-Entrapment" in the "Reason for Animal Behaviour" column. 
#That value is not valid for that column but it is a valid entry for "Cause of Animal Behaviour"
#which was missing for this row/incident entry. Moving to "Cause" column and listing "Reason" as missing. 

#Assigning Entangle-Entrapment to Cause column for one row that has that value in Reason column
Animals["Cause of Animal Health Status"].loc[Animals["Reason for Animal Behaviour"]== "Entangle-Entrapment"]

#Replacing Entangle-Entrapment with a missing value as that is not a valid entry for Reason column. 
Animals["Reason for Animal Behaviour"] = Animals["Reason for Animal Behaviour"].replace({"Entangle-Entrapment": ""})

#Replace Not applicable value with correction "Not Applicable"
Animals["Reason for Animal Behaviour"] = Animals["Reason for Animal Behaviour"].replace({"Not applicable": "Not Applicable"})

#All other values that are not in dictionary are nan (missing) and will be dealt with later after splitting and before modelling. )


In [93]:
#Checking to see how many values do not match values in dictionary.
Animals["Animal Attractant"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Animal Attractant"]).sum()


23005

In [94]:
#Shows how many are False, therefore how many activity types are not in the dictionary. 
Animals.shape[0] - Animals["Animal Attractant"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Animal Attractant"].unique()).sum()
#There are lots of entries that do not match values found in dictionary. Next lines of code will look to see if any can be replaced


50650

In [95]:
#Add column to dataframe that indicates which values match dictionary (True) and which do not (False)
Animals["Animal Attractant_Dict"] = Animals["Animal Attractant"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Animal Attractant"].unique())
Animals.head()


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used,Animal Response to Deterrents,Species Common Name_Dict,Animal Health Status_Dict,Cause of Animal Health Status_Dict,Animal Behaviour_Dict,Reason for Animal Behaviour_Dict,Animal Attractant_Dict
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,Surprise,Prey animal (natural),Presence of Officer/Person,NaN,True,True,False,True,True,True
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,NaN,NaN,NaN,NaN,True,True,True,False,False,False
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,NaN,Prey animal (natural),NaN,NaN,True,True,False,False,False,True
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,NaN,NaN,NaN,NaN,True,True,True,False,False,False
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,0,NaN,NaN,NaN,NaN,Grain,NaN,NaN,True,False,False,False,False,True


In [96]:
#Print values that do not match dictionary to see which need to be replaced. 
Animals["Animal Attractant"][Animals["Animal Attractant_Dict"]== False].unique()

array([nan, 'Domestic animal', 'Not applicable', 'None', 'Other'],
      dtype=object)

In [97]:
#printing all activity types from dictionary to see which best match the errors listed above
DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Animal Attractant"].unique()

array(['Berries (natural)', 'Carrion', 'Compost', 'Domestic Animal',
       'Domestic grass', 'Fish', 'Fruit tree, shrub or garden', 'Garbage',
       'Grain', 'Human food', 'Mate', 'Mineral Lick', 'Not Applicable',
       'Petroleum products', 'Prey animal (natural)', 'Road salt',
       'Small animal feeders', 'Unknown', 'Vegetation (natural)'],
      dtype=object)

In [98]:
Animals["Animal Attractant"][Animals["Animal Attractant"]== "Other"].count()

14

In [99]:
Animals["Animal Attractant"][Animals["Animal Attractant"]== "None"].count()

12

In [100]:
#Replacing "Domestic animal" with correct "Domestic Animal"; "Not applicable" to correct "Not Applicable". 
#I will replace "Other" and "None" with missing values as those entries are not valid. 

Animals["Animal Attractant"] = Animals["Animal Attractant"].replace({"Domestic animal": "Domestic Animal", "Not applicable":"Not Applicable", "Other": "", "None":""})

#Only remaining values that are not in dictionary are the missing values. 

In [101]:
#Checking to see how many values do not match values in dictionary.
Animals["Deterrents Used"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Deterrents/Projectiles used"]).sum()


19540

In [102]:
#Shows how many are False, therefore how many activity types are not in the dictionary. 
Animals.shape[0] - Animals["Deterrents Used"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Deterrents/Projectiles used"].unique()).sum()
#There are lots of entries that do not match values found in dictionary. Next lines of code will look to see if any can be replaced


54115

In [103]:
#Add column to dataframe that indicates which values match dictionary (True) and which do not (False)
Animals["Deterrents Used_Dict"] = Animals["Deterrents Used"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Deterrents/Projectiles used"].unique())
Animals.head()


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,...,Animal Attractant,Deterrents Used,Animal Response to Deterrents,Species Common Name_Dict,Animal Health Status_Dict,Cause of Animal Health Status_Dict,Animal Behaviour_Dict,Reason for Animal Behaviour_Dict,Animal Attractant_Dict,Deterrents Used_Dict
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,...,Prey animal (natural),Presence of Officer/Person,NaN,True,True,False,True,True,True,True
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,...,NaN,NaN,NaN,True,True,True,False,False,False,False
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,...,Prey animal (natural),NaN,NaN,True,True,False,False,False,True,False
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,...,NaN,NaN,NaN,True,True,True,False,False,False,False
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,0,NaN,NaN,NaN,...,Grain,NaN,NaN,True,False,False,False,False,True,False


In [104]:
#Print values that do not match dictionary to see which need to be replaced. 
Animals["Deterrents Used"][Animals["Deterrents Used_Dict"]== False].unique()

array([nan, 'Impact - Electric Shock'], dtype=object)

In [105]:
#printing all activity types from dictionary to see which best match the errors listed above
DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Deterrents/Projectiles used"].unique()

array(['Bear Spray', 'Impact - Beanbag', 'Impact - Chalkball',
       'Impact - Paintball', 'Impact - Pellet', 'Impact - Projectile',
       'Impact - Rubber', 'Lethal Round - Centrefire',
       'Lethal Round - Rimfire', 'Lethal Round - Shotgun',
       'Noise - Banger or Screamer', 'Noise - Blank', 'Noise - Clapping',
       'Noise - Horn', 'Noise - Siren', 'Noise - Voice', 'None',
       'Non-impact - Chalkball', 'Non-impact - Projectile',
       'Not Applicable', 'Other', 'Presence of Officer/Person',
       'Presence of Vehicle', 'Unknown', 'Visual - Flagging or stick'],
      dtype=object)

In [106]:
Animals["Deterrents Used"][Animals["Deterrents Used"]== "Impact - Electric Shock"].count()

4

In [107]:
#There are only 4 occurences of "Impact - Electric Shock" and it is not valid per the dictionary so will replace with missing.

Animals["Deterrents Used"] = Animals["Deterrents Used"].replace({"Impact - Electric Shock": ""})

#The only other values not in dictionary are missing and not to be replaced at this time. 



In [108]:
#Checking to see how many values do not match values in dictionary.
Animals["Animal Response to Deterrents"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Response to Deterrent"]).sum()


10502

In [109]:
#Shows how many are False, therefore how many activity types are not in the dictionary. 
Animals.shape[0] - Animals["Animal Response to Deterrents"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Response to Deterrent"].unique()).sum()
#There are lots of entries that do not match values found in dictionary. Next lines of code will look to see if any can be replaced



63153

In [110]:
#Add column to dataframe that indicates which values match dictionary (True) and which do not (False)
Animals["Animal Response to Deterrents_Dict"] = Animals["Animal Response to Deterrents"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Response to Deterrent"].unique())
Animals.head()


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,...,Deterrents Used,Animal Response to Deterrents,Species Common Name_Dict,Animal Health Status_Dict,Cause of Animal Health Status_Dict,Animal Behaviour_Dict,Reason for Animal Behaviour_Dict,Animal Attractant_Dict,Deterrents Used_Dict,Animal Response to Deterrents_Dict
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,...,Presence of Officer/Person,NaN,True,True,False,True,True,True,True,False
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,...,NaN,NaN,True,True,True,False,False,False,False,False
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,...,NaN,NaN,True,True,False,False,False,True,False,False
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,...,NaN,NaN,True,True,True,False,False,False,False,False
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,0,NaN,NaN,NaN,...,NaN,NaN,True,False,False,False,False,True,False,False


In [111]:
#Print values that do not match dictionary to see which need to be replaced. 
Animals["Animal Response to Deterrents"][Animals["Animal Response to Deterrents_Dict"]== False].unique()

#all values that are not in the dictionary are missing values and not to be replaced at this time. 

array([nan], dtype=object)

In [112]:
#Data cleaning/validation complete for Animals dataset. 

#Drop the columns I added during cleaning that are no longer needed
Animals = Animals.drop(["Species Common Name_Dict", "Animal Health Status_Dict", "Cause of Animal Health Status_Dict", "Animal Behaviour_Dict", "Reason for Animal Behaviour_Dict", "Animal Attractant_Dict", "Deterrents Used_Dict", "Animal Response to Deterrents_Dict"], axis=1)
Animals.head()

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used,Animal Response to Deterrents
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,Surprise,Prey animal (natural),Presence of Officer/Person,NaN
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,NaN,NaN,NaN,NaN
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,NaN,Prey animal (natural),NaN,NaN
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,NaN,NaN,NaN,NaN
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,0,NaN,NaN,NaN,NaN,Grain,NaN,NaN


In [113]:
###
#Next, we're looking for duplicate occurances of the incident number to ensure our final compiled data has UNIQUE incident numbers. 


In [114]:
Animals_subset = Animals[["Incident Number", "Incident Date", "Field Unit", "Protected Heritage Area", "Incident Type"]]
#Duplicate subset includes Incident Type attribute. This is what leads to me to dig deeper into why the Incident Types vary for the "Incidents" Dataset and finding the Null values.
duplicate_Animals_subset = Animals_subset.duplicated(keep=False)
sum(duplicate_Animals_subset)

18314

In [115]:
duplicate_Animals_Inc_Num = Animals.duplicated(subset="Incident Number", keep=False)
sum(duplicate_Animals_Inc_Num)

18314

In [116]:
sum(duplicate_Animals_Inc_Num)==sum(duplicate_Animals_subset)


True

In [117]:
#There are several duplicates of incident numbers here. There are also several attributes that dependent on the value in "Species Common Name" and I do not want to lose any of that data. 
#I will add a new column to this dataset that combines the Incident Number with the "Species Common Name" to create a UNIQUE identifier. 
#I will join the other 3 datasets to this dataset using the Incident Number (and any other common attributes (like Incident Date, Field Unit, Projected Heritage Area, and maybe Incident Type). 
#So the each occurence of the incident number in the Animal dataset will have the same information imported from the other 3 datasets, but I believe this is the best method for preserving all the data we want (and there is no association between the various Response Types, Incident Types or other information that would allow us to do anything else when joining * I emailed David Gummer to ask and am waiting on a response). 


In [118]:
Animals3 = Animals


In [119]:
Animals3.insert(0, "Duplicate Inc_Num", Animals3.duplicated(subset="Incident Number", keep=False))


In [120]:
ValueCounts = Animals3["Incident Number"].value_counts()

In [121]:
ValueCounts["BAN2013-1151"]

11

In [122]:
Counts = []
for i in Animals3["Incident Number"]:
                Counts.append(ValueCounts[i])

Animals3.insert(0, "Duplicate Counts", Counts)
                    
UniqueCounts = []
for i in Animals3["Incident Number"]:
                if ValueCounts[i] >= 1:
                    UniqueCounts.append(ValueCounts[i])
                    ValueCounts[i] -= 1

In [123]:
Animals3.insert(0, "Unique Counts", UniqueCounts)

In [124]:
#Need to convert "Unique Counts" to string type (from integer type) before i'm able to join it with the string "Incident Number" values.

Animals3["Unique Counts"]= Animals3["Unique Counts"].astype(str)

In [125]:
Animals3.insert(0, "UniqueID", Animals3[["Incident Number", "Unique Counts"]].apply(".".join, axis=1))


In [126]:
#Checking to ensure there are no duplicates in the the UniqueID 
duplicates_UniqueID = Animals3.duplicated(subset="UniqueID", keep=False)
sum(duplicates_UniqueID)


0

In [127]:
Animals3

,UniqueID,Unique Counts,Duplicate Counts,Duplicate Inc_Num,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used,Animal Response to Deterrents
0,BAN2010-0003.3,3,3,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,Surprise,Prey animal (natural),Presence of Officer/Person,NaN
1,BAN2010-0003.2,2,3,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,NaN,NaN,NaN,NaN
2,BAN2010-0003.1,1,3,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,NaN,Prey animal (natural),NaN,NaN
3,JNP2010-0011.1,1,1,False,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,NaN,NaN,NaN,NaN
4,JNP2010-0015.1,1,1,False,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,0,NaN,NaN,NaN,NaN,Grain,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73650,2021-HWC-0574-JASFU-0016.2,2,2,True,2021-HWC-0574-JASFU-0016,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,Elk,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73651,2021-HWC-0574-JASFU-0016.1,1,2,True,2021-HWC-0574-JASFU-0016,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,Elk,1,NaN,NaN,Presence - Wildlife Exclusion Zones,NaN,NaN,NaN,NaN
73652,2021-HWC-1114-YKLLFU-0033.1,1,1,False,2021-HWC-1114-YKLLFU-0033,2021-12-31,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Attractant,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73653,2022-HWC-0574-JASFU-0001.2,2,2,True,2022-HWC-0574-JASFU-0001,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,Bighorn Sheep,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
Incidents = pd.read_csv("/Users/nerdbear/Downloads/5. pca-human-wildlife-coexistence-incidents-detailed-records-2010-2021.csv", encoding = 'cp1252')


In [129]:
Incidents.head()


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Latitude Public,Longitude Public,Within Park,Incident Type,Total Staff Involved,Total Staff Hours
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,51.161093,-115.593386,Yes,Human Wildlife Interaction,1.0,2.33
1,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,53.139120,-117.964219,Yes,Rescued/Recovered/Found Wildlife,1.0,1.00
2,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,53.050492,-118.073612,Yes,Attractant,1.0,2.50
3,JNP2010-0023,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,52.858415,-118.102814,Yes,Rescued/Recovered/Found Wildlife,1.0,3.00
4,JNP2010-0016,2010-01-02,Jasper Field Unit,Jasper National Park of Canada,52.857314,-118.103110,Yes,Rescued/Recovered/Found Wildlife,1.0,0.50


In [130]:
Incidents.shape


(64290, 10)

In [131]:
Incidents.dtypes

Incident Number             object
Incident Date               object
Field Unit                  object
Protected Heritage Area     object
Latitude Public            float64
Longitude Public           float64
Within Park                 object
Incident Type               object
Total Staff Involved       float64
Total Staff Hours          float64
dtype: object

In [132]:
#First we are going to clean the data to ensure valid entries on the string values by comparing them to the data dictionary.

In [133]:
#Checking to see how many values do not match values in dictionary.
Incidents["Field Unit"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Field Unit"]).sum()


64290

In [134]:
#Shows how many are False, therefore how many activity types are not in the dictionary. 
Incidents.shape[0] - Incidents["Field Unit"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Field Unit"].unique()).sum()
#There are no entries that do not match values found in dictionary. No replacements needed


0

In [135]:
#Checking to see how many values do not match values in dictionary.
Incidents["Protected Heritage Area"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Protected Heritage Area"]).sum()


64290

In [136]:
#Shows how many are False, therefore how many activity types are not in the dictionary. 
Incidents.shape[0] - Incidents["Protected Heritage Area"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Protected Heritage Area"].unique()).sum()
#There are no entries that do not match values found in dictionary. No replacements needed


0

In [137]:
#Checking to see how many values do not match values in dictionary.
Incidents["Incident Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Incident Type"]).sum()


64258

In [138]:
#Shows how many are False, therefore how many activity types are not in the dictionary. 
Incidents.shape[0] - Incidents["Incident Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Incident Type"].unique()).sum()
#There are no entries that do not match values found in dictionary. No replacements needed


32

In [139]:
#Add column to dataframe that indicates which values match dictionary (True) and which do not (False)
Incidents["Incident Type_Dict"] = Incidents["Incident Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Incident Type"].unique())
Incidents.head()


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Latitude Public,Longitude Public,Within Park,Incident Type,Total Staff Involved,Total Staff Hours,Incident Type_Dict
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,51.161093,-115.593386,Yes,Human Wildlife Interaction,1.0,2.33,True
1,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,53.139120,-117.964219,Yes,Rescued/Recovered/Found Wildlife,1.0,1.00,True
2,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,53.050492,-118.073612,Yes,Attractant,1.0,2.50,True
3,JNP2010-0023,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,52.858415,-118.102814,Yes,Rescued/Recovered/Found Wildlife,1.0,3.00,True
4,JNP2010-0016,2010-01-02,Jasper Field Unit,Jasper National Park of Canada,52.857314,-118.103110,Yes,Rescued/Recovered/Found Wildlife,1.0,0.50,True


In [140]:
#Print values that do not match dictionary to see which need to be replaced. 
Incidents["Incident Type"][Incidents["Incident Type_Dict"]== False].unique()
#All values that are not in dictionary are missing values and will not be replaced now (will look at closer after splitting and before modelling. 


array([nan], dtype=object)

In [141]:
#Drop the columns I added during cleaning that are no longer needed
Incidents = Incidents.drop(["Incident Type_Dict"], axis=1)
Incidents.head()


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Latitude Public,Longitude Public,Within Park,Incident Type,Total Staff Involved,Total Staff Hours
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,51.161093,-115.593386,Yes,Human Wildlife Interaction,1.0,2.33
1,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,53.139120,-117.964219,Yes,Rescued/Recovered/Found Wildlife,1.0,1.00
2,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,53.050492,-118.073612,Yes,Attractant,1.0,2.50
3,JNP2010-0023,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,52.858415,-118.102814,Yes,Rescued/Recovered/Found Wildlife,1.0,3.00
4,JNP2010-0016,2010-01-02,Jasper Field Unit,Jasper National Park of Canada,52.857314,-118.103110,Yes,Rescued/Recovered/Found Wildlife,1.0,0.50


In [142]:
###
#Next, we're looking for duplicate occurances of the incident number to ensure our final compiled data has UNIQUE incident numbers. 


In [143]:
Inc_subset = Incidents[["Incident Number", "Incident Date", "Field Unit", "Protected Heritage Area", "Latitude Public", "Longitude Public", "Within Park"]] 
duplicate_Inc_subset = Inc_subset.duplicated(keep=False)
sum(duplicate_Inc_subset)

64

In [144]:
duplicate_Inc_Inc_Num = Incidents.duplicated(subset="Incident Number", keep=False)
sum(duplicate_Inc_Inc_Num)

64

In [145]:
sum(duplicate_Inc_Inc_Num)==sum(duplicate_Inc_subset)
##Conclusiong, The "Incident Type" and "Total Staff Involved" and "Total Staff Hours" attributes differ but all other attributes remain consistent when the incident number duplicates. 
#Will further investigate duplicates:

True

In [146]:
dup_bool = Incidents["Incident Number"].duplicated(keep=False)
print (dup_bool)
Dup_Incidents = Incidents[dup_bool]
Dup_Incidents
Dup_Incidents["Incident Type"].isna().sum()

0        False
1        False
2        False
3        False
4        False
         ...  
64285    False
64286    False
64287    False
64288    False
64289    False
Name: Incident Number, Length: 64290, dtype: bool


32

In [147]:
Incidents[Incidents["Incident Type"].isna()]
Incidents["Incident Type"].isna().sum()



32

In [148]:
Incidents["Incident Type"].isna().sum() == Dup_Incidents["Incident Type"].isna().sum()


True

In [149]:
#Number of NA's in "Incident Type" column is the same in the entire dataset as it is in the subsetted duplicate dataset (i.e. all of the NA values occur in rows that are duplicate and they occur in half the duplicate rows). Remove these rows. 
#There are only 32 Incident Types that are NaN and they are the 32 Incidents Types that are duplicate.

#There are a total of 64 duplicate rows and of the 64 duplicate rows, there are 32 missing values. Looking at the outputs generated above, it is clear that duplicated rows have missing values for "Incident Type", "Total Staff Involved" and "Total Staff Hours" so no additional information is being provided by the duplicate rows. 
#If "Incident Type" is Nan AND Incident type is duplicate, I will delete that row. The new dataset with removed rows will be called Incidents2. I will then evaluate the remaining rows for duplicates/missing values to ensure I am left with the data I want (no duplicate rows and the total number of rows remaining will be Incidents.shape[0] - Incidents2.shape[0] = 32)

#There are only 32 Incident Types that are NaN and they are the 32 Incidents Types that are duplicate. 
#Delete these rows

In [150]:
Incidents2 = Incidents[Incidents["Incident Type"].notnull()]


In [151]:
#Checking to confirm there are no duplicates remaining: 
#Looking for duplicates in subset
Inc_subset = Incidents2[["Incident Number", "Incident Date", "Field Unit", "Protected Heritage Area", "Latitude Public", "Longitude Public", "Within Park"]] 


In [152]:
duplicate_Inc_subset = Inc_subset.duplicated(keep=False)
sum(duplicate_Inc_subset)


0

In [153]:
#Looking for duplicates in just Incident Number column.
duplicate_Inc_Inc_Num = Incidents2.duplicated(subset="Incident Number", keep=False)
sum(duplicate_Inc_Inc_Num)


0

In [154]:
#Comparing the two
sum(duplicate_Inc_Inc_Num)==sum(duplicate_Inc_subset)
#Conclusion, there are no duplicate Incident Numbers remaining. 



True

In [155]:
#Confirming there are no NA values remaining in "Incident Type" column of new dataframe:
Incidents2["Incident Type"].isna().sum()
#Conclusion, no missing values remaining in new Incidents2 dataset. Will use this dataset for joining with others.


0

In [156]:
Incidents2

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Latitude Public,Longitude Public,Within Park,Incident Type,Total Staff Involved,Total Staff Hours
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,51.161093,-115.593386,Yes,Human Wildlife Interaction,1.0,2.330000
1,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,53.139120,-117.964219,Yes,Rescued/Recovered/Found Wildlife,1.0,1.000000
2,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,53.050492,-118.073612,Yes,Attractant,1.0,2.500000
3,JNP2010-0023,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,52.858415,-118.102814,Yes,Rescued/Recovered/Found Wildlife,1.0,3.000000
4,JNP2010-0016,2010-01-02,Jasper Field Unit,Jasper National Park of Canada,52.857314,-118.103110,Yes,Rescued/Recovered/Found Wildlife,1.0,0.500000
...,...,...,...,...,...,...,...,...,...,...
64285,2021-HWC-0000-JASFU-2861,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,52.876739,-118.091588,Yes,Human Wildlife Interaction,1.0,0.666667
64286,2021-HWC-0000-JASFU-2862,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,53.093617,-118.030592,Yes,Rescued/Recovered/Found Wildlife,1.0,2.000000
64287,2021-HWC-0574-JASFU-0016,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,52.860896,-118.087098,Yes,Human Wildlife Interaction,1.0,0.166667
64288,2021-HWC-1114-YKLLFU-0033,2021-12-31,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,51.380551,-116.147884,Yes,Attractant,1.0,1.750000


In [157]:
#Cross checking to ensure correct number of rows remain.
#Number of rows in Original Dataset, minus number of NA values in duplicates (32) == Number of final rows in new subset. 
Incidents.shape[0] - 32 == Incidents2.shape[0]


True

In [158]:
Responses = pd.read_csv("/Users/nerdbear/Downloads/6. pca-human-wildlife-coexistence-responses-detailed-records-2010-2021.csv", encoding = 'cp1252')


In [159]:
Responses.head()


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Response Type
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Dispose Carcass
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Investigate Incident
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Monitor - patrol
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Dispose Carcass
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Dispose Carcass


In [160]:
Responses.shape


(82109, 5)

In [161]:
Responses.dtypes


Incident Number            object
Incident Date              object
Field Unit                 object
Protected Heritage Area    object
Response Type              object
dtype: object

In [162]:
#First we are going to clean the data to ensure valid entries on the string values by comparing them to the data dictionary.  

In [163]:
#Checking to see how many values do not match values in dictionary.
Responses["Field Unit"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Field Unit"]).sum()


82109

In [164]:
#Shows how many are False, therefore how many activity types are not in the dictionary. 
Responses.shape[0] - Responses["Field Unit"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Field Unit"].unique()).sum()
#There are no entries that do not match values found in dictionary. No replacements needed

0

In [165]:
#Checking to see how many values do not match values in dictionary.
Responses["Protected Heritage Area"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Protected Heritage Area"]).sum()


82109

In [166]:
#Shows how many are False, therefore how many activity types are not in the dictionary. 
Responses.shape[0] - Responses["Protected Heritage Area"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Protected Heritage Area"].unique()).sum()
#There are no entries that do not match values found in dictionary. No replacements needed

0

In [167]:
#Checking to see how many values do not match values in dictionary.
Responses["Response Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Response Type"]).sum()


79747

In [168]:
#Shows how many are False, therefore how many activity types are not in the dictionary. 
Responses.shape[0] - Responses["Response Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Response Type"].unique()).sum()
#There are no entries that do not match values found in dictionary. No replacements needed


2362

In [169]:
#Add column to dataframe that indicates which values match dictionary (True) and which do not (False)
Responses["Response Type_Dict"] = Responses["Response Type"].isin(DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Response Type"].unique())
Responses.head()


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Response Type,Response Type_Dict
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Dispose Carcass,True
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Investigate Incident,True
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Monitor - patrol,True
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Dispose Carcass,True
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Dispose Carcass,True


In [170]:
#Print values that do not match dictionary to see which need to be replaced. 
Responses["Response Type"][Responses["Response Type_Dict"]== False].unique()


array([nan, 'Monitor'], dtype=object)

In [171]:
#printing all activity types from dictionary to see which best match the errors listed above
DataDictionary["Data_Value"][DataDictionary["Data_Field"]== "Response Type"].unique()

array(['Assist other Agency', 'Assist other Field Unit', 'Assist Visitor',
       'Attractant Management', 'Aversive Conditioning', 'Cancel Permit',
       'Capture and transport to captivity', 'Clean Up', 'Close Area',
       'Close Road', 'Collar', 'Collect Sample', 'Cull', 'Destroy Animal',
       'Disentangle', 'Dispatch other Agency', 'Disperse Wildlife Jam',
       'Dispose Carcass', 'Ear Tag', 'Euthanize', 'Evacuate Visitor',
       'Haze - Hard', 'Haze - Soft', 'Immobilize Animal',
       'Inform Visitor', 'Infrastructure modification',
       'Investigate Incident', 'Issue Prohibited Activity Order',
       'Issue Restricted Activity Order', 'Issue Stop Work Order',
       'Leave on Landscape', 'Mark - microchip', 'Mark - paint',
       'Monitor - Camera', 'Monitor - patrol',
       'Monitor - visitor and staff sighting', 'Necropsy',
       'No response required', 'Not Applicable',
       'Refer incident to other agency', 'Rehabilitate area',
       'Relocate animal (s)', 'Req

In [172]:
Responses["Response Type"][Responses["Response Type"]== "Monitor"].count()
#There are only 2 occurences of the invalid "monitor" value


2

In [173]:
Responses.loc[Responses["Response Type"]== "Monitor"]

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Response Type,Response Type_Dict
51090,2018-HWC-0177-ENFU-0004,2018-09-10,Newfoundland East Field Unit,Terra Nova National Park of Canada,Monitor,False
52549,2019-HWC-0144-NPRFU-0001,2019-01-08,Northern Prairies Field Unit,Prince Albert National Park of Canada,Monitor,False


In [174]:
#No way to know which of the 3 valid "Monitor" options value was intended here so replace both with missing values.
#Replacing values that were mis-entered with their proper type, if none was obvious from data dictionary, placing "Unknown"
Responses["Response Type"] = Responses["Response Type"].replace({"Monitor": ""})
#All other values that don't match dictionary are missing values, will not replace at this stage.


In [175]:
#Drop the columns I added during cleaning that are no longer needed
Responses = Responses.drop(["Response Type_Dict"], axis=1)
Responses.head()

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Response Type
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Dispose Carcass
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Investigate Incident
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Monitor - patrol
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Dispose Carcass
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Dispose Carcass


In [176]:
###
#Next, we're looking for duplicate occurances of the incident number to ensure our final compiled data has UNIQUE incident numbers. 


In [177]:
Resp_subset = Responses[["Incident Number", "Incident Date", "Field Unit", "Protected Heritage Area"]]
duplicate_Resp_subset = Resp_subset.duplicated(keep=False)
sum(duplicate_Resp_subset)


32243

In [178]:
duplicate_Resp_Inc_Num = Responses.duplicated(subset="Incident Number", keep=False)
sum(duplicate_Resp_Inc_Num)


32243

In [179]:
sum(duplicate_Resp_Inc_Num)==sum(duplicate_Resp_subset)
#Where the Incident Number is duplicated, all column values are duplicated except for the Response column. This tells me that some incidents had more than one response. In order to join this dataset with the others, we need the Incident Number to be unique so we are going to modify this dataset so that there are no duplicate Incident Numbers. We will do this by keeping the first listing of each incident number, and moving all subsequent responses for duplicate incidents numbers to a new column called "Second Response Type" and 


True

In [180]:
#Finding unique Response Types. *** Emailed David Gummer about whether there is a reference table for which incident types or animal health status's require which response type.
Responses["Response Type"].unique()


array(['Dispose Carcass', 'Investigate Incident', 'Monitor - patrol', nan,
       'Inform Visitor', 'Destroy Animal', 'Request assistance - police',
       'Relocate animal (s)', 'Trap or snare', 'Necropsy',
       'Refer incident to other agency', 'Haze - Soft', 'Clean Up',
       'Traffic control', 'Dispatch other Agency',
       'Issue Restricted Activity Order', 'Close Area', 'Not Applicable',
       'Request assistance - other Agency', 'Immobilize Animal',
       'Leave on Landscape', 'Warning signs', 'Assist other Agency',
       'Collect Sample', 'Assist Visitor', 'No response required',
       'Haze - Hard', 'Capture and transport to captivity', 'Ear Tag',
       'Disperse Wildlife Jam', 'Evacuate Visitor',
       'Aversive Conditioning', 'Close Road',
       'Issue Prohibited Activity Order', 'Euthanize',
       'Infrastructure modification', 'Disentangle',
       'Monitor - visitor and staff sighting', 'Assist other Field Unit',
       'Cull', 'Monitor - Camera', 'Attractant 

In [181]:
#Checking how many of the duplicates have NA values in "Response Type"
dup_Responses = Responses[duplicate_Resp_subset]
dup_Responses


,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Response Type
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Dispose Carcass
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Investigate Incident
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Monitor - patrol
8,PRN2010-0001,2010-01-02,Coastal British Columbia Field Unit,Pacific Rim National Park Reserve of Canada,NaN
9,PRN2010-0001,2010-01-02,Coastal British Columbia Field Unit,Pacific Rim National Park Reserve of Canada,Investigate Incident
...,...,...,...,...,...
82075,2021-HWC-1075-CBCFU-0051,2021-12-20,Coastal British Columbia Field Unit,Pacific Rim National Park Reserve of Canada,Clean Up
82076,2021-HWC-1075-CBCFU-0051,2021-12-20,Coastal British Columbia Field Unit,Pacific Rim National Park Reserve of Canada,Monitor - patrol
82077,2021-HWC-1075-CBCFU-0051,2021-12-20,Coastal British Columbia Field Unit,Pacific Rim National Park Reserve of Canada,Monitor - visitor and staff sighting
82083,2021-HWC-1075-CBCFU-0052,2021-12-21,Coastal British Columbia Field Unit,Pacific Rim National Park Reserve of Canada,Monitor - patrol


In [182]:
#Checking how many of the duplicates have NA values in "Activity Type"
dup_Responses["Response Type"].isna().sum()


583

In [183]:
#Count number of unique Incident Numbers in duplicates.
dup_Responses["Incident Number"].nunique()


12930

In [184]:
#I would like to encode Response Type so each distinct Response type is it's own column with a binary indicator (0 for no and 1 for yes) if a given Response type was involved in a given incident number.

#Count distinct values in Response Type
Responses["Response Type"].nunique()

encoder = OneHotEncoder(handle_unknown='ignore')
encoder_df = pd.DataFrame(encoder.fit_transform(Responses[["Response Type"]]).toarray())
encoder_df.columns = encoder.get_feature_names_out(["Response Type"])
encoder_df.head()
Responses_encoded = Responses.join(encoder_df)
Responses_encoded.head()
Responses_encoded.drop('Response Type', axis = 1, inplace=True)
Responses_encoded.head()

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Response Type_,Response Type_Assist Visitor,Response Type_Assist other Agency,Response Type_Assist other Field Unit,Response Type_Attractant Management,Response Type_Aversive Conditioning,...,Response Type_Rehabilitate area,Response Type_Relocate animal (s),Response Type_Request assistance - other Agency,Response Type_Request assistance - police,Response Type_Traffic control,Response Type_Translocate,Response Type_Trap or snare,Response Type_Unable to respond,Response Type_Warning signs,Response Type_nan
0,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [185]:
#Viewing sums of each response type column. 
print(Responses_encoded[Responses_encoded.columns[4:53]].sum()) 

Response Type_                                            2.0
Response Type_Assist Visitor                            238.0
Response Type_Assist other Agency                       291.0
Response Type_Assist other Field Unit                     9.0
Response Type_Attractant Management                     373.0
Response Type_Aversive Conditioning                     134.0
Response Type_Capture and transport to captivity         66.0
Response Type_Clean Up                                 1006.0
Response Type_Close Area                                860.0
Response Type_Close Road                                197.0
Response Type_Collar                                     33.0
Response Type_Collect Sample                            578.0
Response Type_Cull                                      179.0
Response Type_Destroy Animal                            720.0
Response Type_Disentangle                               128.0
Response Type_Dispatch other Agency                     138.0
Response

In [186]:
##** I would like to merge all columns relating to Reponse Types (columns 4-53) across duplicates on an Incident Numbe so that there are unique Incident Numbers for this dataset.
Responses2 = Responses_encoded[Responses_encoded.columns[4:53]].groupby([Responses['Incident Number'], Responses["Incident Date"], Responses["Field Unit"], Responses["Protected Heritage Area"]]).apply(sum).reset_index()
Responses2

,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Response Type_,Response Type_Assist Visitor,Response Type_Assist other Agency,Response Type_Assist other Field Unit,Response Type_Attractant Management,Response Type_Aversive Conditioning,...,Response Type_Rehabilitate area,Response Type_Relocate animal (s),Response Type_Request assistance - other Agency,Response Type_Request assistance - police,Response Type_Traffic control,Response Type_Translocate,Response Type_Trap or snare,Response Type_Unable to respond,Response Type_Warning signs,Response Type_nan
0,2017-HWC-0005-YKLLFU-0001,2017-08-01,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-HWC-0005-YKLLFU-0002,2017-09-07,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-HWC-0005-YKLLFU-0003,2017-07-08,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-HWC-0005-YKLLFU-0004,2017-06-23,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2017-HWC-0005-YKLLFU-0006,2017-06-28,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62791,YNP2016-0146,2016-10-28,"Lake Louise, Yoho and Kootenay Field Unit",Yoho National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62792,YNP2016-0147,2016-10-30,"Lake Louise, Yoho and Kootenay Field Unit",Yoho National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62793,YNP2016-0148,2016-11-22,"Lake Louise, Yoho and Kootenay Field Unit",Yoho National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62794,YNP2016-0151,2016-12-27,"Lake Louise, Yoho and Kootenay Field Unit",Yoho National Park of Canada,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [187]:
#Confirming whether the new dataset has any duplicate incident numbers
duplicate_Resp2_Inc_Num = Responses2.duplicated(subset="Incident Number", keep=False)
sum(duplicate_Resp2_Inc_Num)


0

In [188]:

#Cross checking to ensure correct number of rows remain.
#Number of rows in Original Dataset, minus (number of rows in duplicates subset minus number of UNIQUE rows in duplicates subset) == Number of final rows in new subset. 
Responses.shape[0] - (dup_Responses.shape[0] - dup_Responses["Incident Number"].nunique()) == Responses2.shape[0]



True

In [189]:
#(In other words, I want to ensure that our new dataset has the same number of Unique incident numbers as our original dataset)
Responses["Incident Number"].nunique() == Responses2["Incident Number"].nunique()

#Conclusion, correct number of rows are remaining in our new dataset.



True

In [190]:
#Joining datasets without losing any rows from any dataset.

#Checking all 4 datasets and comparing Incident Numbers. Because we'll be using Animals dataset as our main one to join the others into,
#I Want to see if there are any incident numbers included in the other 3 datasets that are not already in the Animals dataset.
#Conclusion based on results below, there are three (3) incident numbers included in other datasets that do not exist in Animals.

AnimalIDs = Animals3["Incident Number"].unique()
AnimalIDs
AnimalIDs = np.sort(AnimalIDs)
AnimalIDs
AnimalIDs.size
ActivityIDs = Activities2["Incident Number"]
ActivityIDs
ActivityIDs = np.sort(ActivityIDs)
ActivityIDs
ActivityIDs.size
dif1 = list(set(ActivityIDs)-set(AnimalIDs))
dif1
IncidentIDs = Incidents2["Incident Number"]
IncidentIDs.size
IncidentIDs = np.sort(IncidentIDs)
IncidentIDs
dif2 = list(set(IncidentIDs)-set(AnimalIDs))
dif2
ResponseIDs = Responses2["Incident Number"]
ResponseIDs.size
ResponseIDs = np.sort(ResponseIDs)
ResponseIDs
dif3 = list(set(ResponseIDs)-set(AnimalIDs))
dif3
print(dif1, dif2, dif3)


['2021-VS-0748-YKLLFU-0001', 'PEINP2011-0131'] ['2021-VS-0748-YKLLFU-0001', '2019-HWC-0000-BANFU-1457', 'PEINP2011-0131'] ['PEINP2011-0131']


In [191]:
Animals.head()

,UniqueID,Unique Counts,Duplicate Counts,Duplicate Inc_Num,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used,Animal Response to Deterrents
0,BAN2010-0003.3,3,3,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,Surprise,Prey animal (natural),Presence of Officer/Person,NaN
1,BAN2010-0003.2,2,3,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,NaN,NaN,NaN,NaN
2,BAN2010-0003.1,1,3,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,NaN,Prey animal (natural),NaN,NaN
3,JNP2010-0011.1,1,1,False,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,NaN,NaN,NaN,NaN
4,JNP2010-0015.1,1,1,False,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,0,NaN,NaN,NaN,NaN,Grain,NaN,NaN


In [192]:

#Now joining datasets together.
#Doing Outer Joins to ensure no loss of data at this stage for Incident Numbers that exist in other datasets but not in the Animals datset we are joining to.

JoinedData1 = pd.merge(Animals3, Activities2, how="outer", on = ["Incident Number", "Incident Date", "Field Unit", "Protected Heritage Area"])
JoinedData1


,UniqueID,Unique Counts,Duplicate Counts,Duplicate Inc_Num,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,...,Activity Type_Swimming - Coastal,Activity Type_Swimming - Facilities,Activity Type_Swimming - Flat Water,Activity Type_Swimming - Swiftwater,Activity Type_Townsite Activity,Activity Type_Tram/Ski Lift/Gondola,Activity Type_Tubing / River Drifting,Activity Type_Unknown,Activity Type_Via-Ferrata,Activity Type_nan
0,BAN2010-0003.3,3,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,BAN2010-0003.2,2,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,BAN2010-0003.1,1,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,JNP2010-0011.1,1,1.0,False,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JNP2010-0015.1,1,1.0,False,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73652,2021-HWC-1114-YKLLFU-0033.1,1,1.0,False,2021-HWC-1114-YKLLFU-0033,2021-12-31,"Lake Louise, Yoho and Kootenay Field Unit",Banff National Park of Canada,Attractant,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73653,2022-HWC-0574-JASFU-0001.2,2,2.0,True,2022-HWC-0574-JASFU-0001,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,Bighorn Sheep,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73654,2022-HWC-0574-JASFU-0001.1,1,2.0,True,2022-HWC-0574-JASFU-0001,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,Bighorn Sheep,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73655,NaN,NaN,NaN,NaN,2021-VS-0748-YKLLFU-0001,2021-06-19,Banff Field Unit,Banff National Park of Canada,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [193]:
#Confirming that Incident Numbers contained in Activities but not in Animals dataset were still joined into the new dataset. 
JoinedData1.loc[JoinedData1["Incident Number"].isin(dif1)]


,UniqueID,Unique Counts,Duplicate Counts,Duplicate Inc_Num,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Species Common Name,...,Activity Type_Swimming - Coastal,Activity Type_Swimming - Facilities,Activity Type_Swimming - Flat Water,Activity Type_Swimming - Swiftwater,Activity Type_Townsite Activity,Activity Type_Tram/Ski Lift/Gondola,Activity Type_Tubing / River Drifting,Activity Type_Unknown,Activity Type_Via-Ferrata,Activity Type_nan
73655,NaN,NaN,NaN,NaN,2021-VS-0748-YKLLFU-0001,2021-06-19,Banff Field Unit,Banff National Park of Canada,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73656,NaN,NaN,NaN,NaN,PEINP2011-0131,2011-07-08,Prince Edward Island Field Unit,Prince Edward Island National Park of Canada,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [194]:

JoinedData2 = pd.merge(JoinedData1, Incidents2, how="outer", on = ["Incident Number", "Incident Date", "Field Unit", "Protected Heritage Area"])
JoinedData2


,UniqueID,Unique Counts,Duplicate Counts,Duplicate Inc_Num,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type_x,Species Common Name,...,Activity Type_Tubing / River Drifting,Activity Type_Unknown,Activity Type_Via-Ferrata,Activity Type_nan,Latitude Public,Longitude Public,Within Park,Incident Type_y,Total Staff Involved,Total Staff Hours
0,BAN2010-0003.3,3,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Coyote,...,0.0,0.0,0.0,1.0,51.161093,-115.593386,Yes,Human Wildlife Interaction,1.0,2.33
1,BAN2010-0003.2,2,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Elk,...,0.0,0.0,0.0,1.0,51.161093,-115.593386,Yes,Human Wildlife Interaction,1.0,2.33
2,BAN2010-0003.1,1,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,Wolf,...,0.0,0.0,0.0,1.0,51.161093,-115.593386,Yes,Human Wildlife Interaction,1.0,2.33
3,JNP2010-0011.1,1,1.0,False,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,White-tailed Deer,...,0.0,0.0,0.0,0.0,53.139120,-117.964219,Yes,Rescued/Recovered/Found Wildlife,1.0,1.00
4,JNP2010-0015.1,1,1.0,False,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,None,...,0.0,0.0,0.0,0.0,53.050492,-118.073612,Yes,Attractant,1.0,2.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73653,2022-HWC-0574-JASFU-0001.2,2,2.0,True,2022-HWC-0574-JASFU-0001,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,Bighorn Sheep,...,0.0,0.0,0.0,0.0,53.162687,-117.964186,Yes,Human Wildlife Interaction,1.0,0.50
73654,2022-HWC-0574-JASFU-0001.1,1,2.0,True,2022-HWC-0574-JASFU-0001,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,Bighorn Sheep,...,0.0,0.0,0.0,0.0,53.162687,-117.964186,Yes,Human Wildlife Interaction,1.0,0.50
73655,NaN,NaN,NaN,NaN,2021-VS-0748-YKLLFU-0001,2021-06-19,Banff Field Unit,Banff National Park of Canada,NaN,NaN,...,0.0,0.0,0.0,0.0,NaN,NaN,Yes,Highway Fence Intrusion,1.0,1.00
73656,NaN,NaN,NaN,NaN,PEINP2011-0131,2011-07-08,Prince Edward Island Field Unit,Prince Edward Island National Park of Canada,NaN,NaN,...,0.0,0.0,0.0,0.0,46.496335,-63.406292,Yes,Rescued/Recovered/Found Wildlife,1.0,0.33


In [195]:
JoinedData2["Incident Type_x"] == JoinedData2["Incident Type_y"]

0         True
1         True
2         True
3         True
4         True
         ...  
73653     True
73654     True
73655    False
73656    False
73657    False
Length: 73658, dtype: bool

In [196]:
#Both Animals3 and Incidents2 contained a column for "Incident Type" so joining the two created columns "_x" and "_y"
#Looking for differences between the two columns. 

In [197]:
difference = list(set(JoinedData2["Incident Type_x"]) - set(JoinedData2["Incident Type_y"]))

In [198]:
difference

[nan]

In [199]:
JoinedData2["Incident Type_x"].isna().sum()

3

In [200]:
JoinedData2["Incident Type_y"].isna().sum()

0

In [201]:
#Conclusion, Incident Type_x column contains 3 na values, whereas Incident Type_y contains none. 
#Will drop "Incident Type_x". 

JoinedData2.drop('Incident Type_x', axis = 1, inplace=True)
JoinedData2.head()

,UniqueID,Unique Counts,Duplicate Counts,Duplicate Inc_Num,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Species Common Name,Sum of Number of Animals,...,Activity Type_Tubing / River Drifting,Activity Type_Unknown,Activity Type_Via-Ferrata,Activity Type_nan,Latitude Public,Longitude Public,Within Park,Incident Type_y,Total Staff Involved,Total Staff Hours
0,BAN2010-0003.3,3,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Coyote,2.0,...,0.0,0.0,0.0,1.0,51.161093,-115.593386,Yes,Human Wildlife Interaction,1.0,2.33
1,BAN2010-0003.2,2,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Elk,1.0,...,0.0,0.0,0.0,1.0,51.161093,-115.593386,Yes,Human Wildlife Interaction,1.0,2.33
2,BAN2010-0003.1,1,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Wolf,3.0,...,0.0,0.0,0.0,1.0,51.161093,-115.593386,Yes,Human Wildlife Interaction,1.0,2.33
3,JNP2010-0011.1,1,1.0,False,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,White-tailed Deer,1.0,...,0.0,0.0,0.0,0.0,53.139120,-117.964219,Yes,Rescued/Recovered/Found Wildlife,1.0,1.00
4,JNP2010-0015.1,1,1.0,False,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,None,0.0,...,0.0,0.0,0.0,0.0,53.050492,-118.073612,Yes,Attractant,1.0,2.50


In [202]:
#Moving columns around so key information is closer to start of dataframe and all the activity types are at the end. 
eight = JoinedData2.pop('Incident Type_y')
JoinedData2.insert(8, 'Incident Type', eight)

In [203]:
nine = JoinedData2.pop('Latitude Public')
JoinedData2.insert(9, 'Latitude Public', nine)

In [204]:
ten = JoinedData2.pop('Longitude Public')
JoinedData2.insert(10, 'Longitude Public', ten)

In [205]:
eleven = JoinedData2.pop('Within Park')
JoinedData2.insert(11, 'Within Park', eleven)

In [206]:
twelve = JoinedData2.pop('Total Staff Involved')
JoinedData2.insert(12, 'Total Staff Involved', twelve)

In [207]:
thirteen = JoinedData2.pop('Total Staff Hours')
JoinedData2.insert(13, 'Total Staff Hours', thirteen)

In [208]:
JoinedData2.head()

,UniqueID,Unique Counts,Duplicate Counts,Duplicate Inc_Num,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Latitude Public,...,Activity Type_Swimming - Coastal,Activity Type_Swimming - Facilities,Activity Type_Swimming - Flat Water,Activity Type_Swimming - Swiftwater,Activity Type_Townsite Activity,Activity Type_Tram/Ski Lift/Gondola,Activity Type_Tubing / River Drifting,Activity Type_Unknown,Activity Type_Via-Ferrata,Activity Type_nan
0,BAN2010-0003.3,3,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,BAN2010-0003.2,2,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,BAN2010-0003.1,1,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,JNP2010-0011.1,1,1.0,False,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,53.139120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JNP2010-0015.1,1,1.0,False,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,53.050492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [209]:
#Confirming that Incident Numbers contained in Incidents but not in Animals dataset were still joined into the new dataset. 
JoinedData2.loc[JoinedData2["Incident Number"].isin(dif2)]



,UniqueID,Unique Counts,Duplicate Counts,Duplicate Inc_Num,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Latitude Public,...,Activity Type_Swimming - Coastal,Activity Type_Swimming - Facilities,Activity Type_Swimming - Flat Water,Activity Type_Swimming - Swiftwater,Activity Type_Townsite Activity,Activity Type_Tram/Ski Lift/Gondola,Activity Type_Tubing / River Drifting,Activity Type_Unknown,Activity Type_Via-Ferrata,Activity Type_nan
73655,NaN,NaN,NaN,NaN,2021-VS-0748-YKLLFU-0001,2021-06-19,Banff Field Unit,Banff National Park of Canada,Highway Fence Intrusion,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73656,NaN,NaN,NaN,NaN,PEINP2011-0131,2011-07-08,Prince Edward Island Field Unit,Prince Edward Island National Park of Canada,Rescued/Recovered/Found Wildlife,46.496335,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73657,NaN,NaN,NaN,NaN,2019-HWC-0000-BANFU-1457,2019-08-20,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
JoinedData3 = pd.merge(JoinedData2, Responses2, how="outer", on = ["Incident Number", "Incident Date", "Field Unit", "Protected Heritage Area"])
JoinedData3


,UniqueID,Unique Counts,Duplicate Counts,Duplicate Inc_Num,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Latitude Public,...,Response Type_Rehabilitate area,Response Type_Relocate animal (s),Response Type_Request assistance - other Agency,Response Type_Request assistance - police,Response Type_Traffic control,Response Type_Translocate,Response Type_Trap or snare,Response Type_Unable to respond,Response Type_Warning signs,Response Type_nan
0,BAN2010-0003.3,3,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BAN2010-0003.2,2,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BAN2010-0003.1,1,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JNP2010-0011.1,1,1.0,False,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,53.139120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JNP2010-0015.1,1,1.0,False,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,53.050492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73653,2022-HWC-0574-JASFU-0001.2,2,2.0,True,2022-HWC-0574-JASFU-0001,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,53.162687,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73654,2022-HWC-0574-JASFU-0001.1,1,2.0,True,2022-HWC-0574-JASFU-0001,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,53.162687,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73655,NaN,NaN,NaN,NaN,2021-VS-0748-YKLLFU-0001,2021-06-19,Banff Field Unit,Banff National Park of Canada,Highway Fence Intrusion,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73656,NaN,NaN,NaN,NaN,PEINP2011-0131,2011-07-08,Prince Edward Island Field Unit,Prince Edward Island National Park of Canada,Rescued/Recovered/Found Wildlife,46.496335,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [211]:
#Confirming that Incident Numbers contained in Responses but not in Animals dataset were still joined into the new dataset. 
JoinedData3.loc[JoinedData3["Incident Number"].isin(dif3)]



,UniqueID,Unique Counts,Duplicate Counts,Duplicate Inc_Num,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Latitude Public,...,Response Type_Rehabilitate area,Response Type_Relocate animal (s),Response Type_Request assistance - other Agency,Response Type_Request assistance - police,Response Type_Traffic control,Response Type_Translocate,Response Type_Trap or snare,Response Type_Unable to respond,Response Type_Warning signs,Response Type_nan
73656,NaN,NaN,NaN,NaN,PEINP2011-0131,2011-07-08,Prince Edward Island Field Unit,Prince Edward Island National Park of Canada,Rescued/Recovered/Found Wildlife,46.496335,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [212]:
#Renaming our final complete Dataset.
CompleteData = JoinedData3
CompleteData

,UniqueID,Unique Counts,Duplicate Counts,Duplicate Inc_Num,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Latitude Public,...,Response Type_Rehabilitate area,Response Type_Relocate animal (s),Response Type_Request assistance - other Agency,Response Type_Request assistance - police,Response Type_Traffic control,Response Type_Translocate,Response Type_Trap or snare,Response Type_Unable to respond,Response Type_Warning signs,Response Type_nan
0,BAN2010-0003.3,3,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BAN2010-0003.2,2,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BAN2010-0003.1,1,3.0,True,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JNP2010-0011.1,1,1.0,False,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,53.139120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JNP2010-0015.1,1,1.0,False,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,53.050492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73653,2022-HWC-0574-JASFU-0001.2,2,2.0,True,2022-HWC-0574-JASFU-0001,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,53.162687,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73654,2022-HWC-0574-JASFU-0001.1,1,2.0,True,2022-HWC-0574-JASFU-0001,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,53.162687,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73655,NaN,NaN,NaN,NaN,2021-VS-0748-YKLLFU-0001,2021-06-19,Banff Field Unit,Banff National Park of Canada,Highway Fence Intrusion,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73656,NaN,NaN,NaN,NaN,PEINP2011-0131,2011-07-08,Prince Edward Island Field Unit,Prince Edward Island National Park of Canada,Rescued/Recovered/Found Wildlife,46.496335,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [213]:
CompleteData.drop('Unique Counts', axis = 1, inplace=True)
CompleteData.drop('Duplicate Counts', axis = 1, inplace=True)
CompleteData.drop('Duplicate Inc_Num', axis = 1, inplace=True)
CompleteData.head()

,UniqueID,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Latitude Public,Longitude Public,Within Park,Total Staff Involved,...,Response Type_Rehabilitate area,Response Type_Relocate animal (s),Response Type_Request assistance - other Agency,Response Type_Request assistance - police,Response Type_Traffic control,Response Type_Translocate,Response Type_Trap or snare,Response Type_Unable to respond,Response Type_Warning signs,Response Type_nan
0,BAN2010-0003.3,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,-115.593386,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BAN2010-0003.2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,-115.593386,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BAN2010-0003.1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,-115.593386,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JNP2010-0011.1,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,53.139120,-117.964219,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JNP2010-0015.1,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,53.050492,-118.073612,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [214]:
CompleteData['UniqueID'] = CompleteData['UniqueID'].fillna(CompleteData['Incident Number'])

In [215]:
CompleteData

,UniqueID,Incident Number,Incident Date,Field Unit,Protected Heritage Area,Incident Type,Latitude Public,Longitude Public,Within Park,Total Staff Involved,...,Response Type_Rehabilitate area,Response Type_Relocate animal (s),Response Type_Request assistance - other Agency,Response Type_Request assistance - police,Response Type_Traffic control,Response Type_Translocate,Response Type_Trap or snare,Response Type_Unable to respond,Response Type_Warning signs,Response Type_nan
0,BAN2010-0003.3,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,-115.593386,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BAN2010-0003.2,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,-115.593386,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BAN2010-0003.1,BAN2010-0003,2010-01-01,Banff Field Unit,Banff National Park of Canada,Human Wildlife Interaction,51.161093,-115.593386,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JNP2010-0011.1,JNP2010-0011,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Rescued/Recovered/Found Wildlife,53.139120,-117.964219,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JNP2010-0015.1,JNP2010-0015,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Attractant,53.050492,-118.073612,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73653,2022-HWC-0574-JASFU-0001.2,2022-HWC-0574-JASFU-0001,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,53.162687,-117.964186,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73654,2022-HWC-0574-JASFU-0001.1,2022-HWC-0574-JASFU-0001,2021-12-31,Jasper Field Unit,Jasper National Park of Canada,Human Wildlife Interaction,53.162687,-117.964186,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73655,2021-VS-0748-YKLLFU-0001,2021-VS-0748-YKLLFU-0001,2021-06-19,Banff Field Unit,Banff National Park of Canada,Highway Fence Intrusion,NaN,NaN,Yes,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73656,PEINP2011-0131,PEINP2011-0131,2011-07-08,Prince Edward Island Field Unit,Prince Edward Island National Park of Canada,Rescued/Recovered/Found Wildlife,46.496335,-63.406292,Yes,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [219]:
CompleteData.to_csv("/Users/nerdbear/Downloads/Complete_HWC_Data.csv")